# Codes for DST-data etc.

In [ ]:
import requests
import numpy as np
import pandas as pd

def get_data(table_id,variables):
    base = 'https://api.statbank.dk/v1/data/{id}/JSONSTAT?lang=en'.format(id = table_id)
    
    for var in variables:
        base += '&{v}'.format(v = var) 

    response=requests.get(base)
    data_json=response.json()
    return data_json


## RAS201 data

In [ ]:
data5=get_data('RAS201',['Tid=*','OMRÅDE=085','SOCIO=50']) # Sjælland
data4=get_data('RAS201',['OMRÅDE=084','Tid=*','SOCIO=50']) # Hovedstaden
data3=get_data('RAS201',['Tid=*','OMRÅDE=083','SOCIO=50']) # Syddanmark
data2=get_data('RAS201',['Tid=*','OMRÅDE=082','SOCIO=50']) # Midtjylland
data1=get_data('RAS201',['Tid=*', 'OMRÅDE=081','SOCIO=50']) # Nordjylland

indexlabels=['2008','2009','2010','2011','2012','2013','2014','2015','2016']

df1=pd.DataFrame(data1['dataset']['value'],index=indexlabels,columns=['Region Nordjylland'])
df2=pd.DataFrame(data2['dataset']['value'],index=indexlabels,columns=['Region Midtjylland'])
df3=pd.DataFrame(data3['dataset']['value'],index=indexlabels,columns=['Region Syddanmark'])
df4=pd.DataFrame(data4['dataset']['value'],index=indexlabels,columns=['Region Hovedstaden'])
df5=pd.DataFrame(data5['dataset']['value'],index=indexlabels,columns=['Region Sjælland'])

dfs=[df1,df2,df3,df4,df5]
RAS201=pd.concat(dfs,axis=1)
RAS201=df_Unemployment_by_residence.reindex(index=RAS201.index[::-1]) # Reversing row index
RAS201

## NAN1 data

In [ ]:
indexlabels_bnp=['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
data_bnp=get_data('NAN1',['Tid=2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017','TRANSAKT=B1GQK','PRISENHED=LAN_M']) # Nordjylland
BNP=pd.DataFrame(data_bnp['dataset']['value'],index=indexlabels_bnp,columns=['2010-priser, kædede værdier, (mia kr.)'])
#BNP=BNP.reindex(index=BNP.index[::-1])
#BNP=BNP.transpose()
BNP

## FOLK1A data (non-API)

In [ ]:
path='https://github.com/Naeim-S/GRP33/blob/master/EXAM/Dataset/FOLK1APython.xlsx?raw=true'

FOLK1A=pd.read_excel(path)



## Dataframe to markdown

In [ ]:
def df_to_markdown(*dfs, sep_line='\n---\n', **kwargs):
    """Convert pandas dataframe to markdown table."""
    import tabulate

    disable_numparse = kwargs.pop('disable_numparse', True)
    tablefmt = kwargs.pop('tablefmt', 'pipe')
    headers = kwargs.pop('headers', 'keys')
    
    for df in dfs:
        print(tabulate.tabulate(df, tablefmt=tablefmt, headers=headers,
                                disable_numparse=disable_numparse, **kwargs))
        if sep_line is not None:
            print(sep_line)
            
            
df_to_markdown(FOLK1A,BNP,RAS201)

# Visualizations for descriptive analysis

# Visualizations for analysis

In [ ]:
import pandas as pd
import time, requests, re
import matplotlib.pyplot as plt


import numpy as np
import seaborn as sns 
sns.set(rc={'figure.figsize':(11.7,8.27)})
plt.style.use('ggplot')
# sns.set_style("whitegrid", {'axes.grid' : True})

%matplotlib inline

In [ ]:
raw='https://raw.githubusercontent.com/Naeim-S/GRP33/master/EXAM/merged_ratio.csv'
merged=pd.read_csv(raw)
del merged['Unnamed: 0']

sektor_order=merged.sektor.unique()
area_order=merged.area.unique()

merged.tail(10)

list(merged)

## Elasticity for sector and area

In [ ]:
#Exclude "danmark" and sort by n_area and year (for nicer plot):
#merged_sektor = merged.query('sektor').sort_values(by=["ratio_sektor","year"], ascending=False)

sns.set_palette("tab10")
sns.relplot(x="year", y="RR_sektor", 
            hue="sektor", 
            kind="line", 
            height=7, 
            aspect=1.5, 
            legend = "brief",
            hue_order=sektor_order,
            data=merged)
plt.ylabel("First differences ratio to BNP in sectors")
#plt.yticks(range(0,100,10))
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('1_RR_Sector.png',bbox_inches='tight')
# All jobs for each area over time

In [ ]:
#Exclude "danmark" and sort by n_area and year (for nicer plot):
#merged_area = merged.query(' area != "danmark"').sort_values(by=["ratio_area","year"], ascending=False)

sns.set_palette("tab10")
sns.relplot(x="year", y="RR_area", 
            hue="area", 
            kind="line", 
            height=7, 
            aspect=1.5, 
            legend = "brief",
            hue_order=area_order,
            data=merged)
plt.ylabel("Elasticity")
#plt.yticks(range(0,100,10))
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('2_RR_Area.png',bbox_inches='tight')


# All jobs for each area over time

## Sectors across areas

In [ ]:
#All sectors for each area over time

sns.set_palette("tab10")

#KØBENHAVN
sns.relplot(x="year", y="RR_sektor_area", 
        hue="sektor", 
        kind="line", 
        legend = "brief",        
        height=7, 
        aspect=1.5, 
        hue_order=sektor_order,
        data=merged.query('area == "region hovedstaden"').sort_values(by=["year", "ratio_sektor_area"], ascending=False))
plt.title("Region Capital Area")
plt.ylabel("Elasticity")

plt.savefig('3.1_RR_sector_area.png',bbox_inches='tight')

#MIDTJYLLAND
sns.relplot(x="year", y="RR_sektor_area", 
        hue="sektor", 
        kind="line", 
        height=7, 
        aspect=1.5, 
        legend = "brief",
        hue_order=sektor_order,
        data=merged.query('area == "region-midtjylland" ').sort_values(by=["year", "ratio_sektor_area"], ascending=False))
plt.title("Region Central Jutland")
plt.ylabel("Elasticity")

plt.savefig('3.2_RR_sector_area.png',bbox_inches='tight')

#SYDJYLLAND
sns.relplot(x="year", y="RR_sektor_area", 
        hue="sektor", 
        kind="line",
        height=7, 
        aspect=1.5, 
        legend = "brief",
        hue_order=sektor_order,
        data=merged.query('area == "region syddanmark" ').sort_values(by=["year", "ratio_sektor_area"], ascending=False))
plt.title("Region Southern Denmark")
plt.ylabel("Elasticity")

plt.savefig('3.3_RR_sector_area.png',bbox_inches='tight')

#REGION SJÆLLAND
sns.relplot(x="year", y="RR_sektor_area", 
        hue="sektor", 
        kind="line", 
        height=7, 
        aspect=1.5, 
        legend = "brief",
        hue_order=sektor_order,
        data=merged.query('area == "region-sjaelland" ').sort_values(by=["year", "ratio_sektor_area"], ascending=False))
plt.title("Region Zealand")
plt.ylabel("Elasticity")

plt.savefig('3.4_RR_sector_area.png',bbox_inches='tight')


#NORDJYLLAND
sns.relplot(x="year", y="RR_sektor_area", 
        hue="sektor", 
        kind="line", 
        height=7, 
        aspect=1.5, 
        legend = "brief",
        hue_order=sektor_order,
        data=merged.query('area == "region-nordjylland" ').sort_values(by=["year", "ratio_sektor_area"], ascending=False))
plt.title("Region Northern Jutland")
plt.ylabel("Elasticity")

plt.savefig('3.5_RR_sector_area.png',bbox_inches='tight')


## Jobtypes for sectors

In [ ]:
#KØBENHAVN IT - minus it-courses
sns.relplot(x="year", y="RR_jobtype_area", 
        hue="jobtype", 
        kind="line", 
        legend = "brief",        
        height=3.5, 
        aspect=1.2, 
      #  ax=ax1,
      #  hue_order=sektor_order,
        data=merged.query('area == "region hovedstaden" & sektor == "it" & jobtype != "itkurser"').sort_values(by=["year", "RR_jobtype_area"], ascending=False))
plt.title("a) IT in Capital Area")
plt.ylabel("Elasticity")
#plt.ylim(0,0.04)
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('4.1_RR_jobtype_area.png',bbox_inches='tight')


#Sjælland industri 
sns.relplot(x="year", y="RR_jobtype_area", 
        hue="jobtype", 
        kind="line", 
        legend = "brief",        
        height=3.5, 
        aspect=1.2, 
      #  ax=ax1,
      #  hue_order=sektor_order,
        data=merged.query('area == "region-sjaelland" & sektor == "industri"').sort_values(by=["year", "RR_jobtype_area"], ascending=False))
plt.title("c) Industry in Zealand")
plt.ylabel("Elasticity")
#plt.ylim(0,0.04)
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('4.2_RR_jobtype_area.png',bbox_inches='tight')

#KØBENHAVN kontor - minus office-courses
sns.relplot(x="year", y="RR_jobtype_area", 
        hue="jobtype", 
        kind="line", 
        legend = "brief",        
        height=3.5, 
        aspect=1.2, 
      #  ax=ax1,
      #  hue_order=sektor_order,
        data=merged.query('area == "region hovedstaden" & sektor == "kontor" & jobtype != "kontorkurser"').sort_values(by=["year", "RR_jobtype_area"], ascending=False))
plt.title("b) Office work in Capital Area")
plt.ylabel("Elasticity")
#plt.ylim(0,0.04)
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('4.3_RR_jobtype_area.png',bbox_inches='tight')


#Sjælland handel
sns.relplot(x="year", y="RR_jobtype_area", 
        hue="jobtype", 
        kind="line", 
        legend = "brief",        
        height=3.5, 
        aspect=1.2, 
      #  ax=ax1,
      #  hue_order=sektor_order,
        data=merged.query('area == "region-sjaelland" & sektor == "handel"').sort_values(by=["year", "RR_jobtype_area"], ascending=False))
plt.title("d) Trade in Zealand")
plt.ylabel("Elasticity")
#plt.ylim(0,0.04)
plt.xlim(2007,2017)
plt.xticks(range(2007,2018,2))
plt.xlabel("Year")

plt.savefig('4.4_RR_jobtype_area.png',bbox_inches='tight')
